# Load packages

In [ ]:
import os
import langchain
langchain.__version__

'0.1.0'

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
def load_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

api_key = load_api_key('api_key.txt')

In [ ]:
os.environ["OPENAI_API_KEY"] = api_key
llm = ChatOpenAI()
llm.invoke("한국어 어순이 어떻게 돼?")

AIMessage(content='한국어의 어순은 주어-동사-목적어(주격보다는 목적격을 중심으로) 순서를 따르는 주순이 기본적인 어순입니다. 하지만 한국어는 조사가 많이 사용되는 언어이기 때문에 어순이 유연하게 바뀔 수도 있습니다. 때로는 주어나 목적어가 먼저 나올 수도 있고, 동사가 먼저 나올 수도 있습니다. 이러한 유연성으로 인해 한국어는 다양한 어순을 가지고 있습니다.')

# Basics

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 언어학자야. 한 번 대화를 잘 완성시켜 봐."),
    ("user", "{input}")
])

chain = prompt | llm
chain.invoke({"input": "안락사를 주제로 두 사람이 대화 하는 거 턴 3번에 걸쳐서 만들어 줘."})

AIMessage(content='대화 1:\n\n언어학자: 안락사에 대해 어떻게 생각하시나요?\n\n상대방: 안락사는 정말 복잡한 문제라고 생각해요. 인간의 삶과 죽음에 대한 깊은 고민이 필요한 주제죠.\n\n언어학자: 네, 그렇군요. 안락사는 환자의 고통을 덜어주기 위한 선택이지만, 도덕적인 쟁점이 많이 존재하죠.\n\n대화 2:\n\n언어학자: 안락사를 결정할 때 어떤 요소가 고려되어야 한다고 생각하시나요?\n\n상대방: 안락사 결정은 환자의 의사와 병원, 가족 등 다양한 요소가 모여서 이뤄져야 한다고 생각해요. 모든 측면을 고려해야 한다고 봐요.\n\n언어학자: 네, 그렇군요. 환자의 의사와 가족의 의견을 중요하게 고려하는 것은 안락사 결정에서 매우 중요한 부분이죠.\n\n대화 3:\n\n언어학자: 안락사에 대한 사회적 태도는 어떠한지 궁금해요. 여러 견해가 많이 나뉘어지는 주제입니다.\n\n상대방: 네, 안락사에 대한 사회적 태도는 정말 다양하죠. 종교적, 윤리적, 법적 쟁점들이 얽혀있어서 많은 논의가 필요한 주제입니다.\n\n언어학자: 맞아요. 안락사는 다양한 측면에서 논의되고 있는 주제이며, 사회적 태도 또한 계속해서 변화하고 있습니다. 이에 대한 고찰이 필요한 분야입니다.')

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "안락사를 주제로 두 사람이 대화 하는 거 턴 3번에 걸쳐서 만들어 줘."})

'대화 1:\n\nA: 안락사에 대해 생각해 봤어?\n\nB: 네, 안락사는 정말 복잡한 문제야. 어떤 경우에는 필요한 선택이지만 어떤 경우에는 도덕적인 고민이 될 수도 있어.\n\n대화 2:\n\nA: 그렇지. 안락사를 선택하는 것이 그 사람의 삶과 죽음에 대한 개인적인 결정이라는 점을 잊으면 안 돼.\n\nB: 맞아. 그 사람의 삶을 존중하면서, 그들의 선택을 이해하고 지지해 주어야 한다는 거지.\n\n대화 3:\n\nA: 그래, 그들이 고통을 견딜 수 없다면, 안락사가 마지막으로 남은 선택일 수도 있지.\n\nB: 네, 우리는 그 결정을 비난하거나 판단하기보다는 그들의 고통을 이해하고 공감해 주어야 해.\n\nA: 정말로 그렇지. 안락사는 어려운 주제지만, 서로 존중하고 대화하면서 이해하려 노력해야겠어.'

# Retrieval

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
text = f"""
A: 병원에서 그냥 이렇게 거기 뭐 인공호흡기로 연명한다는 것은 난 나는 반대야.
그렇다고 해서 그 존엄사를 선택을 한다고 해서 보통 기본적인 수분하고 수분하고 영양은 기본적으로 주거든.
고통을 느끼는 진통제 정도는 투여가 되는 거거든. 근데 나는 내가 만약에 한다면 나는 선 그 생명 연장을 하지 말아라라고 나는 말할 거야.
오빠 생각은 어때?

B: 내가 실제 경험했던 얘기를 해 줄게.
우리 부친이 돌아가실 때 충남대 병원에서 그 의사가 딱 얘기를 하더라고.마음의 준비를 하시라.
말을 못 해도 되는데 생명 연장은 할 수가 있다. 그러니까 그 얘기를 딱 하면서 그것이 뭐냐 그랬더니 기도 삽관이라는 거래.
어떤 호스를 집어넣어 가지고 인공적으로 호흡 그 호흡을 자가호흡이 아니라
그래서 가족들한테 그 얘기를 딱 물어봤을 때 어느 가족이 그냥 돌아가시게 내버려 둘 수가 있나.
장을 해 달라 말씀은 못 사셔도 못 하셔도 연장을 해 달라 결국은 그렇게 이제 결론을 내려 가지고 이틀을 더 사셨어.
이틀 동안 그냥 아무 말씀 안 하시고 그냥 눈만 뜨시 뜨고 계시다가 그냥 돌아가셨는데 어 지금 와서 생각을 해 보니까
그것이 사실은 가족들한테 훨씬 더 괴로움을 많이 준 거고 어 가족들한테 어 치명적인 머릿속에 지워지지 않는 어떤 그
그 뭐라고 할까 그 머릿속에 정말로 안 지워지더라고 이거는 끝까지. 내가 죽을 때까지 그 그 모습은 끝까지 남아 있을 것 같아.

"""


In [ ]:
#docs = [text] # URL만 작동하는듯

In [ ]:
''''
# FAISS(Facebook AI Similarity Search)로 문서를 벡터화
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(docs, embeddings)
'''

"'\n# FAISS(Facebook AI Similarity Search)로 문서를 벡터화\nfrom langchain_community.vectorstores import FAISS\nfrom langchain.text_splitter import RecursiveCharacterTextSplitter\n\ntext_splitter = RecursiveCharacterTextSplitter()\ndocuments = text_splitter.split_documents(docs)\nvector = FAISS.from_documents(docs, embeddings)\n"

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""해당 질문에 대해서 각 화자의 speaking style and tone을 고려해서, 또 각자 세 번의 turn을 만들어서 대화를 완성시켜 줘. 중첩되는 정보 없어야 해.

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

from langchain_core.documents import Document

document_chain.invoke({
    "input": "해당 질문에 대해서 각 화자의 speaking style and tone을 고려해서, 또 각자 세 번의 turn을 만들어서 대화를 완성시켜 줘. 중첩되는 정보 없어야 해.",
    "context": [Document(page_content=text)]
})

'A: 그러면 당신은 어떻게 생각하시나요? 그런 상황이 와 닥쳤을 때 당신은 어떤 선택을 할 거라고 생각하시나요?\nB: 저는 개인적으로는 존엄사를 선택할 것 같아요. 가족들에게 더 큰 고통을 주지 않기 위해서라도 그렇게 선택할 거예요.\nA: 그렇군요. 제 생각에도 가족들을 위해서라도 존엄사를 선택하는 것이 더 나을 것 같아요. 그렇게 결정하는 게 어려운 일이겠지만 말이죠.'

# 대화형 LangChain

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

# Zero shot test

In [ ]:
examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [ ]:
def load_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

api_key = load_api_key('api_key.txt')

In [ ]:
class AIUtility:
    openai.api_key = os.getenv("OPENAI_KEY")

    def __init__(self, prompt, temperature=0, model="gpt-3.5-turbo"):
        self.prompt = prompt
        self.temperature = temperature
        self.model = model

    def __str__(self):
        return f"{self.prompt}"

    def get_completion(self):
        messages = [{"role": "user", "content": self.prompt}]
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=messages,
            temperature=self.temperature
        )
        return response.choices[0].message["content"]

    def print_completion(self):
        response = self.get_completion()
        print(response)
        print("-----------------------------------------------------------------")

In [ ]:
class ZeroShotUtility:
    def __init__(self, template, temperature=0):
        self.template = template
        self.temperature = temperature

    def __str__(self):
        return f"{self.template}"

    def print_travel_modes(self, question):
        prompt_template = ChatPromptTemplate.from_template(self.template)
        message = prompt_template.format_messages(question=question)
        llm = ChatOpenAI(temperature=self.temperature,
                         openai_api_key=os.getenv("OPENAI_KEY"))
        response = llm(message)
        print(response.content)
        print("-----------------------------------------------------------------")

In [ ]:
def get_prompt_template():
    return """
    I want you to play the role of a travel bot and answer the question
    Return the answer in a json format
    you should have all the options listed in an array structure.
    The root of the array should be named as "options"
    You should fill up the values yourself
    remove non-viable or non-feasible options from the json

    Question: {question}
    """

In [ ]:
if __name__ == '__main__':
    prompt_template = zeroshot.ZeroShot.get_prompt_template()

    zeroShot = zeroshotutility.ZeroShotUtility(template=prompt_template)
    question = "How to reach Jersey City from Hoboken?"
    zeroShot.print_travel_modes(question=question)

# Fewshot test

In [ ]:
class FewShot:

    @staticmethod
    def get_examples():
        examples = [
            {
                "question": "How to reach Kolkata airport to Kolkata Zoo?",
                "answer": """
                option 1: mode=bus, min_time_in_min=75, max_time_in_min=90, description=take bus no 37 from airport. It
                will drop you in alipore. Take a walk of 5 mins
                option 2: mode=metro, min_time_in_min=40, max_time_in_min=60, description=take metro rail from airport. It
                will drop you in joka. Take a walk of 10 mins
                option 3: mod=train, min_time_in_min=45, max_time_in_min=60, description=take local train from airport. It
                will drop you in new alipore. Take a walk of 10 mins
                option 4: mod=walk, min_time_in_min=400, max_time_in_min=500, description=take Take a walk of 500 mins
                """
            },
            {
                "question": "How to reach Hyderabad airport to Hyderabad HiTech City?",
                "answer": """
                option 1: mode=bus, min_time_in_min=60, max_time_in_min=75, description=take bus no 23 from airport. It
                will drop you in hitech city.
                option 2: mode=car, min_time_in_min=30, max_time_in_min=45, description=take ola/uber/airport shuttle.
                It will drop you in HiTech city
                option 3: mode=motorbike, min_time_in_min=25, max_time_in_min=40. description=take motorbike.It will
                drop you in HiTech city
                """
            },
            {
                "question": "How to reach Heathrow airport to Hounslow Central?",
                "answer": """
                        option 1: mode=tube, min_time_in_min=10, max_time_in_min=15. description=take tube from airport.
                        It will frop you in Hounslow central
                        option 2: mode=car, min_time_in_min=25, max_time_in_min=35. description=take rental car from
                         airport . It will drop you in Hounslow Central
                        """
            }
        ]
        return examples

    @staticmethod
    def get_example_template():
        template = """
        Question: {question}
        Answer: {answer}
        """
        example_variables = ["question", "answer"]
        return template, example_variables

    @staticmethod
    def get_prefix():
        return f"""
        I want you to play the role of a travel bot and answer the question
        Return the answer in a json format
        you should have all the options listed in an array structure.
        The root of the array should be named as "options"
        You should fill up the values yourself
        remove non-viable or non-feasible options from the json
        """

    @staticmethod
    def get_suffix():
        return """
                Question: {question}
                """

In [ ]:
if __name__ == '__main__':
    examples = fewshot.FewShot.get_examples()
    prefix = fewshot.FewShot.get_prefix()
    suffix = fewshot.FewShot.get_suffix()
    example_template, example_variables = fewshot.FewShot.get_example_template()

    fewShot = fewshotutility.FewShotUtility(examples=examples,
                                            prefix=prefix,
                                            suffix=suffix,
                                            input_variables=["question"],
                                            example_template=example_template,
                                            example_variables=example_variables
                                            )
    question = "How to reach Jersey City from Hoboken?"
    prompt = fewShot.get_prompt(question)
    fewShot.print_travel_modes(prompt)

In [ ]:
chat_model = ChatOpenAI(openai_api_key=api_key)

In [ ]:
class ZeroShotUtility:
    def __init__(self, template, temperature=0):
        self.template = template
        self.temperature = temperature

    def __str__(self):
        return f"{self.template}"

    def print_travel_modes(self, question):
        prompt_template = ChatPromptTemplate.from_template(self.template)
        message = prompt_template.format_messages(question=question)
        llm = ChatOpenAI(temperature=self.temperature,
                         openai_api_key=os.getenv("OPENAI_KEY"))
        response = llm(message)
        print(response.content)
        print("-----------------------------------------------------------------")